In [1]:
import time
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score

In [2]:
train_data = pd.read_csv('/Users/arponbiswas/Deep-Learning-Prejects/NeuroCalculator/Dataset/Train_Dataset/train_dataset.csv')
test_data = pd.read_csv('/Users/arponbiswas/Deep-Learning-Prejects/NeuroCalculator/Dataset/Test_Dataset/test_dataset.csv')

In [3]:
train_data

,num1,num2,sum
0,0,0,0
1,0,1,1
2,0,2,2
3,0,3,3
4,0,4,4
5,0,5,5
6,0,6,6
7,0,7,7
8,0,8,8
9,0,9,9


In [4]:
torch.tensor(train_data[['num1', 'num2']].values)[0].shape

torch.Size([2])

In [5]:
len(torch.tensor(train_data['sum'].values))

42

In [6]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.x = torch.tensor(data[['num1', 'num2']].values).float()
        self.y = torch.tensor(data['sum'].values).unsqueeze(1).float()
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    def __len__(self):
        return len(self.y)

In [7]:
train_ds = CustomDataset(train_data)
test_ds = CustomDataset(test_data)

In [8]:
train_dl = DataLoader(train_ds, batch_size=5, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=5, shuffle=True)

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.name = 'Model_1'
        self.fc1 = nn.Linear(2, 1)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return x

In [10]:
model = Model()
summary(model, input_size=(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 1]                    --
├─Linear: 1-1                            [1, 1]                    3
├─ReLU: 1-2                              [1, 1]                    --
Total params: 3
Trainable params: 3
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

### What if out is 5.999999

- if out is in range (5.5, 6.5) it's 6
- if out is in range (4.5, 5.5) it's 5

### Use torch.round() for that

In [11]:
def Training(model, epochs=10):
    writer = SummaryWriter(log_dir=f'/Users/arponbiswas/Deep-Learning-Prejects/NeuroCalculator/Tensorboard-Graph/runs/{model.name}')
    writer.add_graph(model, input_to_model=torch.tensor([[2., 3.]]))
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    print('#'*100)
    print('Training and Testing is started.')
    print('='*100)
    for epoch in tqdm(range(epochs), desc='Epoch Progress'):
        #time.sleep(0.2)
        train_losses = []
        test_losses = []
        train_acc = []
        test_acc = []
        train_preds = []
        test_preds = []
        #Training
        model.train()
        for x, y in train_dl:
            optimizer.zero_grad()
            out = model(x)
            loss = loss_fn(out, y)
            loss.backward()
            optimizer.step()
            pred = torch.tensor([[int(i)] for i in out])
            train_losses.append(loss.item())
            train_acc.extend(item.item() for item in y)
            train_preds.extend(item.item() for item in pred)
        #Testing
        model.eval()
        with torch.no_grad():
            for x, y in test_dl:
                out = model(x)
                loss = loss_fn(out, y)
                pred = torch.tensor([[int(i)] for i in out])
                test_losses.append(loss.item())
                test_acc.extend(item.item() for item in y)
                test_preds.extend(item.item() for item in pred)
        #Metrics
        train_accuracy = accuracy_score(train_acc, train_preds)
        test_accuracy = accuracy_score(test_acc, test_preds)
        train_mse = mean_squared_error(train_acc, train_preds)
        test_mse = mean_squared_error(test_acc, test_preds)
        train_mae = mean_absolute_error(train_acc, train_preds)
        test_mae = mean_absolute_error(test_acc, test_preds)
        train_r2 = r2_score(train_acc, train_preds)
        test_r2 = r2_score(test_acc, test_preds)
        train_loss = sum(train_losses) / len(train_losses)
        test_loss = sum(test_losses) / len(test_losses)
        #Tensorboard
        writer.add_scalar('Train/Accuracy', train_accuracy, epoch)
        writer.add_scalar('Train/MSE', train_mse, epoch)
        writer.add_scalar('Train/MAE', train_mae, epoch)
        writer.add_scalar('Train/R2', train_r2, epoch)
        writer.add_scalar('Train/Loss', train_loss, epoch)
        writer.add_scalar('Test/Accuracy', test_accuracy, epoch)
        writer.add_scalar('Test/MSE', test_mse, epoch)
        writer.add_scalar('Test/MAE', test_mae, epoch)
        writer.add_scalar('Test/R2', test_r2, epoch)
        writer.add_scalar('Test/Loss', test_loss, epoch)

In [12]:
Training(model, epochs=2500)

####################################################################################################
Training and Testing is started.


Epoch Progress: 100%|██████████| 2500/2500 [00:25<00:00, 99.43it/s] 


In [13]:
sample = torch.tensor([
    [580.0, 55.0],
    [455.0, 3435.0],
    [110.0, 10.20],
    [220.0, 20.970],
    [330.50, 30.80],
    [440.0, 40.0],
    [550.0, 50.0],
    [660.0, 60.0],
    [7709.450, 70.0],
    [880.0, 80.0],
    [990.76, 90.0],
    [1000.870, 100.0]
])
model.eval()
with torch.no_grad():
    out = model(sample)
    actual = sample.sum(dim=1, keepdim=True)
    print('Prediction')
    print(out)
    print('Actual')
    print(actual)

Prediction
tensor([[ 635.0000],
        [3890.0000],
        [ 120.2000],
        [ 240.9700],
        [ 361.3000],
        [ 480.0000],
        [ 600.0000],
        [ 720.0000],
        [7779.4502],
        [ 960.0000],
        [1080.7600],
        [1100.8700]])
Actual
tensor([[ 635.0000],
        [3890.0000],
        [ 120.2000],
        [ 240.9700],
        [ 361.3000],
        [ 480.0000],
        [ 600.0000],
        [ 720.0000],
        [7779.4502],
        [ 960.0000],
        [1080.7600],
        [1100.8700]])


In [14]:
scripted_model = torch.jit.script(model)
scripted_model.save('/Users/arponbiswas/Deep-Learning-Prejects/NeuroCalculator/Models/scripted_model.pt')